In [8]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
"""
 Deep2BSDE solver with hard-coded Black-Scholes-Barenblatt equation.
 module : tensorflow 2.5.0 / tensorflow_probability 0.13.0
 - tensorflow 2向けにコードを全体的に修正。
 - 長く推定を続けるとy0の値が不安定になる。
"""

import time, datetime
import tensorflow as tf
import numpy as np
from tensorflow.python.training import moving_averages
import tensorflow_probability as tfp

start_time = time.time()

name = 'BSB_TF2_ADJ'
d = 100
batch_size = 64
T = 1.0
N = 20
h = T/N
sqrth = np.sqrt(h)
n_maxstep = 10000
n_displaystep = 50
Xinit = np.array([1.0,0.5]*int(d/2))
mu = 0
sigma = 0.4
sigma_min = 0.1
sigma_max = 0.4
r = 0.05


def sigma_value(W):
    return sigma_max * \
        tf.cast(tf.greater_equal(W, tf.cast(0,tf.float32)),
        tf.float32) + \
            sigma_min * tf.cast(tf.greater(tf.cast(0,tf.float32), W),
            tf.float32)

def f_tf(t, X, Y, Z, Gamma):
    return -0.5*tf.expand_dims(tf.linalg.trace(
        tf.square(tf.expand_dims(X,-1)) * \
            (tf.square(sigma_value(Gamma))) * Gamma),-1) + \
                r * (Y - tf.reduce_sum(X*Z, 1, keepdims = True))

def g_tf(X):
    return tf.reduce_sum(tf.square(X),1, keepdims = True)

class Dense(tf.Module):
    def __init__(self, input_dim, output_size, activation, is_last, name=None):
        super(Dense, self).__init__(name=name)
        with self.name_scope:
            self.w = tf.Variable(
                tf.random.normal([input_dim, output_size],mean=0,stddev=5/np.sqrt(input_dim+output_size)), name='w')
            #self.b = tf.Variable(tf.zeros([output_size]), name='b')
            self.BN = tf.keras.layers.BatchNormalization()
            self.activation = activation
            self.is_last = is_last
    @tf.Module.with_name_scope
    def __call__(self, x):
        y = tf.matmul(x, self.w) #+ self.b
        y = self.BN(y)
        if self.activation == "ReLU":
            y = tf.nn.relu(y)
        return y

class MLP(tf.Module):
    def __init__(self, input_size, sizes, activations, name=None):
        super(MLP, self).__init__(name=name)
        self.layers = []
        with self.name_scope:
            self.BN = tf.keras.layers.BatchNormalization()
            self.layers.append(self.BN)
            for _k, (size, act_fun) in enumerate(zip(sizes,activations)):
                self.layers.append(Dense(input_dim=input_size, 
                                         output_size=size,
                                         activation=act_fun,
                                         is_last=(_k == (len(sizes)-1)),
                                         name="Dence"+str(_k)
                                         ))
                input_size = size
    @tf.Module.with_name_scope
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class forward(tf.Module):
    def __init__(self,name=None):
        super(forward, self).__init__(name=name)
        self.mlps = []
        with self.name_scope:
            for t in range(N-1):
                mlps = []
                mlps.append(MLP(input_size=d, sizes=[d,d,d],activations=["ReLU","ReLU","Linear"],name="A"+str(t)))
                mlps.append(MLP(input_size=d, sizes=[d,d,d**2],activations=["ReLU","ReLU","Linear"],name="G"+str(t)))
                self.mlps.append(mlps)
    @tf.Module.with_name_scope
    def __call__(self, X, Y, Z, A, Gamma):
        for t,mlps in zip(range(N-1),self.mlps):            
            sigma_ = sigma_value(tf.reshape(tf.linalg.trace(Gamma), [batch_size, 1]))            
            dW = tf.random.normal(shape=[batch_size, d], stddev = 1, dtype=tf.float32)            
            # Y update inside the loop
            dX = mu * X * h + sqrth * sigma_ * X * dW
            Y = Y + f_tf(t * h, X, Y, Z, Gamma)*h \
                    + tf.reshape(tf.linalg.trace(Gamma), [batch_size, 1])*tf.square(sigma_)*(X**2)* h*0.5 \
                    + tf.reduce_sum(Z*dX, 1, keepdims = True)                    
            Z = Z + A * h + tf.squeeze(tf.matmul(Gamma, tf.expand_dims(dX, -1), transpose_b=False))            
            X = X + dX            
            A = mlps[0](X)/d
            Gamma = mlps[1](X)/d**2            
            Gamma = tf.reshape(Gamma, [batch_size, d, d])
            
        sigma_ = sigma_value(tf.reshape(tf.linalg.trace(Gamma), [batch_size, 1]))
        
        dW = tf.random.normal(shape=[batch_size, d], stddev = 1, dtype=tf.float32)
            
        # Y update inside the loop
        dX = mu * X * h + sqrth * sigma_ * X * dW

        Y = Y + f_tf((N-1) * h, X, Y, Z, Gamma)*h \
                + tf.reshape(tf.linalg.trace(Gamma), [batch_size, 1])*tf.square(sigma_)*(X**2)* h*0.5 \
                + tf.reduce_sum(Z*dX, 1, keepdims = True)

        X = X + dX
            
        loss = tf.reduce_mean(tf.square(Y-g_tf(X)))

        return loss


class Deep2BSDE(tf.Module):
    def __init__(self, name=None):
        super(Deep2BSDE, self).__init__(name=name)
        with self.name_scope:
            self.X = tf.Variable(np.ones([batch_size, d]) * Xinit,
                                dtype=tf.float32,
                                trainable=False)
            self.Y0 = tf.Variable(tf.random.uniform([1],
                                                    minval=0, maxval=50, dtype=tf.float32),
                                name='Y0')
            self.Z0 = tf.Variable(tf.random.uniform([1, d],
                                                    minval=-.1, maxval=.1,
                                                    dtype=tf.float32),
                                name='Z0')
            self.Gamma0 = tf.Variable(tf.random.uniform([d,d],
                                                minval=-1, maxval=1,
                                                dtype=tf.float32),
                                    name='Gamma0')
            self.A0 = tf.Variable(tf.random.uniform([1, d], 
                                                    minval=-.1, maxval=.1,
                                                    dtype=tf.float32), 
                                name='A0')
            self.fwd = forward()
    @tf.Module.with_name_scope
    def __call__(self):
        allones = tf.ones(shape=[batch_size, 1], 
                            dtype=tf.float32,
                            name='MatrixOfOnes')
        Y = allones * self.Y0
        Z = tf.matmul(allones, self.Z0)
        A = tf.matmul(allones, self.A0)
        
        Gamma = tf.multiply(tf.ones(shape=[batch_size, d, d],
                                        dtype=tf.float32), 
                                self.Gamma0)
        
        loss = self.fwd(self.X,Y,Z,A,Gamma)
        return loss
    

model = Deep2BSDE()
model()
print(len(model.trainable_variables))


422


In [9]:
model.trainable_variables

(<tf.Variable 'deep2_bsde/A0:0' shape=(1, 100) dtype=float32, numpy=
 array([[-0.00978825, -0.05037732, -0.02390485,  0.05539054, -0.01309607,
          0.08556928, -0.08436184,  0.02476323,  0.00565634,  0.03113887,
         -0.00672381,  0.01221917, -0.07407641,  0.06672163, -0.02426369,
          0.0368259 ,  0.0903944 , -0.09100068,  0.05847735,  0.09092829,
          0.04615016,  0.05299927,  0.03836348, -0.03313522, -0.02403133,
         -0.07058623,  0.06732497,  0.00817807, -0.07207272,  0.03877018,
         -0.03359105,  0.08953183,  0.05497266, -0.05561788,  0.07567101,
         -0.00623412, -0.06745782, -0.03410008, -0.09127209, -0.02915487,
         -0.06438057, -0.01033721,  0.00056069, -0.07023072, -0.03824253,
         -0.02350364,  0.06696544, -0.02943134,  0.00424957, -0.09196001,
         -0.02080412,  0.00295651, -0.01272607, -0.08528676,  0.06695586,
          0.02066283,  0.07398953, -0.02286191, -0.04721377, -0.00608204,
         -0.09758864, -0.08480299, -0.02784

In [10]:
name_list = []
for f in model.trainable_variables:
    name_list.append(f.name)

In [11]:
len(name_list)

422

In [5]:
name_list

['deep2_bsde/A0:0',
 'deep2_bsde/Gamma0:0',
 'deep2_bsde/Y0:0',
 'deep2_bsde/Z0:0',
 'deep2_bsde/forward/A0/batch_normalization/gamma:0',
 'deep2_bsde/forward/A0/batch_normalization/beta:0',
 'deep2_bsde/forward/A0/Dence0/w:0',
 'deep2_bsde/forward/A0/Dence0/batch_normalization_1/gamma:0',
 'deep2_bsde/forward/A0/Dence0/batch_normalization_1/beta:0',
 'deep2_bsde/forward/A0/Dence1/w:0',
 'deep2_bsde/forward/A0/Dence1/batch_normalization_2/gamma:0',
 'deep2_bsde/forward/A0/Dence1/batch_normalization_2/beta:0',
 'deep2_bsde/forward/A0/Dence2/w:0',
 'deep2_bsde/forward/G0/batch_normalization_4/gamma:0',
 'deep2_bsde/forward/G0/batch_normalization_4/beta:0',
 'deep2_bsde/forward/G0/Dence0/w:0',
 'deep2_bsde/forward/G0/Dence0/batch_normalization_5/gamma:0',
 'deep2_bsde/forward/G0/Dence0/batch_normalization_5/beta:0',
 'deep2_bsde/forward/G0/Dence1/w:0',
 'deep2_bsde/forward/G0/Dence1/batch_normalization_6/gamma:0',
 'deep2_bsde/forward/G0/Dence1/batch_normalization_6/beta:0',
 'deep2_bsde/

In [7]:
151/2/19

3.973684210526316

In [30]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
"""
 Beck et al. 2019 Appendix A.3 Original Program
 Deep2BSDE solver with hard-coded Black-Scholes-Barenblatt equation.
 元のプログラムは、tensorflow 1.15(GCPで構築したTF1.15で検証)でもTable 6のように収束しない。
 修正点：Originalのプログラムに".compat.v1"を加えTF2.0対応　+　f/gamma/sigmaの式の修正 : line 31/44-55/158-180 + 初期値の変更: line 134 (0-1 -> 0-100)　
"""

import time, datetime
import tensorflow as tf
import numpy as np
from tensorflow.python.training import moving_averages

start_time = time.time()
tf.compat.v1.reset_default_graph()

name = 'BSB_TF2'
d = 100
batch_size = 64
T = 1.0
N = 20
h = T/N
sqrth = np.sqrt(h)
n_maxstep = 3000
n_displaystep = 100
n_neuronForA = [d,d,d,d]
n_neuronForGamma = [d,d,d,d**2]
Xinit = np.array([1.0,0.5]*50)
mu = 0
#sigma = 0.4
sigma_min = 0.1
sigma_max = 0.4
r = 0.05

_extra_train_ops = []

def sigma_value(W):
    return sigma_max * \
        tf.cast(tf.greater_equal(W, tf.cast(0,tf.float64)),
            tf.float64) + \
        sigma_min * tf.cast(tf.greater(tf.cast(0,tf.float64), W),
            tf.float64)

            
""" 
def f_tf(t, X, Y, Z, Gamma):
    return -0.5*tf.expand_dims(tf.compat.v1.trace(
        tf.square(tf.expand_dims(X,-1)) * \
            (tf.square(sigma_value(Gamma))-sigma**2) * Gamma),-1) + \
                r * (Y - tf.compat.v1.reduce_sum(X*Z, 1, keep_dims = True)) 
"""
def f_tf(t, X, Y, Z, Gamma):
    return -0.5*tf.expand_dims(tf.compat.v1.trace(
        tf.square(tf.expand_dims(X,-1)) * \
            (tf.square(sigma_value(Gamma))) * Gamma),-1) + \
                r * (Y - tf.compat.v1.reduce_sum(X*Z, 1, keep_dims = True)) 


def g_tf(X):
    return tf.compat.v1.reduce_sum(tf.square(X),1, keep_dims = True)

def sigma_function(X):
    return sigma * tf.matrix_diag(X)

def mu_function(X):
    return mu * X


def _one_time_net(x, name, isgamma = False):
    with tf.compat.v1.variable_scope(name):
        x_norm = _batch_norm(x, name='layer0_normalization')
        layer1 = _one_layer(x_norm, (1-isgamma)*n_neuronForA[1] \
            +isgamma*n_neuronForGamma[1], name = 'layer1')
        layer2 = _one_layer(layer1, (1-isgamma)*n_neuronForA[2] \
            +isgamma*n_neuronForGamma[2], name = 'layer2')
        z = _one_layer(layer2, (1-isgamma)*n_neuronForA[3] \
            +isgamma*n_neuronForGamma[3], activation_fn=None,
            name = 'final')
    return z

def _one_layer(input_, output_size, activation_fn=tf.nn.relu,stddev=5.0, name='linear'):
    
    with tf.compat.v1.variable_scope(name):
        shape = input_.get_shape().as_list()
        w = tf.compat.v1.get_variable('Matrix', [shape[1], output_size],
        tf.float64,
        tf.compat.v1.random_normal_initializer(
                    stddev=stddev/np.sqrt(shape[1]+output_size)))
        hidden = tf.matmul(input_, w)
        hidden_bn = _batch_norm(hidden, name='normalization')
    if activation_fn:
        return activation_fn(hidden_bn)
    else:
        return hidden_bn

def _batch_norm(x, name):
    """Batch normalization"""
    with tf.compat.v1.variable_scope(name):
        params_shape = [x.get_shape()[-1]]
        beta = tf.compat.v1.get_variable('beta', params_shape, tf.float64,
                initializer=tf.compat.v1.random_normal_initializer(
                    0.0, stddev=0.1, dtype=tf.float64))
        gamma = tf.compat.v1.get_variable('gamma', params_shape, tf.float64,
                initializer=tf.compat.v1.random_uniform_initializer(
                    0.1, 0.5, dtype=tf.float64))
        moving_mean = tf.compat.v1.get_variable('moving_mean',
                params_shape, tf.float64,
                initializer=tf.compat.v1.constant_initializer(0.0, tf.float64),
                trainable=False)
        moving_variance = tf.compat.v1.get_variable('moving_variance',
                params_shape, tf.float64,
                initializer=tf.compat.v1.constant_initializer(1.0, tf.float64),
                trainable=False)
        mean, variance = tf.nn.moments(x, [0], name='moments')
        _extra_train_ops.append(
            moving_averages.assign_moving_average(
                moving_mean, mean, 0.99))
        _extra_train_ops.append(
            moving_averages.assign_moving_average(
                moving_variance, variance, 0.99))
        y = tf.nn.batch_normalization(
            x, mean, variance, beta, gamma, 1e-6)
        y.set_shape(x.get_shape())
        return y

with tf.compat.v1.Session() as sess:

    dW = tf.compat.v1.random_normal(shape=[batch_size, d],
                        stddev = 1, dtype=tf.float64)
    X = tf.Variable(np.ones([batch_size, d]) * Xinit,
                    dtype=tf.float64,
                    trainable=False)
    Y0 = tf.Variable(tf.compat.v1.random_uniform(
                    [1],
                    minval=0, maxval=50, dtype=tf.float64),
                    name='Y0')
    Z0 = tf.Variable(tf.compat.v1.random_uniform(
                    [1, d],
                    minval=-.1, maxval=.1,
                    dtype=tf.float64),
                    name='Z0')
    Gamma0 = tf.Variable(tf.compat.v1.random_uniform(
                    [d, d],
                    minval=-1, maxval=1,
                    dtype=tf.float64), name='Gamma0')
    A0 = tf.Variable(tf.compat.v1.random_uniform(
                    [1, d], minval=-.1, maxval=.1,
                    dtype=tf.float64), name='A0')
    allones = tf.ones(shape=[batch_size, 1], dtype=tf.float64,
                    name='MatrixOfOnes')
    Y = allones * Y0
    Z = tf.matmul(allones, Z0)
    A = tf.matmul(allones, A0)
    Gamma = tf.multiply(tf.ones(shape=[batch_size, d, d],
                    dtype=tf.float64), Gamma0)
    with tf.compat.v1.variable_scope('forward'):
        for t in range(N-1):
            # Y update inside the loop
            sigma_ = sigma_value(tf.reshape(tf.compat.v1.trace(Gamma), [batch_size, 1]))
            dX = mu * X * h + sqrth * sigma_ *X* dW
            Y = Y + f_tf(t * h, X, Y, Z, Gamma)*h \
                            + tf.reshape(tf.compat.v1.trace(Gamma), [batch_size, 1])*tf.square(sigma_)*(X**2)* h*0.5 \
                            + tf.compat.v1.reduce_sum(Z*dX, 1, keep_dims = True)
            # Z update inside the loop
            Z = Z + A * h \
                            + tf.squeeze(tf.matmul(Gamma,
                                        tf.expand_dims(dX, -1),
                                                    transpose_b=False))
            X = X + dX
            A = _one_time_net(X, str(t+1)+"A" )/d
            Gamma = _one_time_net(X, str(t+1)+"Gamma",
                        isgamma = True)/d**2
            Gamma = tf.reshape(Gamma, [batch_size, d, d])
            dW = tf.compat.v1.random_normal(shape=[batch_size, d],
                        stddev = 1, dtype=tf.float64)
        # Y update outside of the loop - terminal time step
        sigma_ = sigma_value(X)
        dX = mu * X * h + sqrth * sigma_ *  dW
        Y = Y + f_tf((N-1) * h, X, Y, Z, Gamma)*h \
                        + tf.reshape(tf.compat.v1.trace(Gamma), [batch_size, 1])*tf.square(sigma_)*(X**2) *h*0.5 \
                        + tf.compat.v1.reduce_sum(Z * dX, 1, keep_dims=True)
        X = X + dX
        loss = tf.reduce_mean(tf.square(Y-g_tf(X)))

    # training operations
    global_step = tf.compat.v1.get_variable('global_step', [],
                initializer=tf.compat.v1.constant_initializer(0),
                trainable=False, dtype=tf.int32)

    learning_rate = tf.compat.v1.train.exponential_decay(1.0, global_step,
                decay_steps = 200,
                decay_rate = 0.5,
                staircase=True)

    trainable_variables = tf.compat.v1.trainable_variables()
    

In [31]:
trainable_variables

[<tf.Variable 'Y0:0' shape=(1,) dtype=float64>,
 <tf.Variable 'Z0:0' shape=(1, 100) dtype=float64>,
 <tf.Variable 'Gamma0:0' shape=(100, 100) dtype=float64>,
 <tf.Variable 'A0:0' shape=(1, 100) dtype=float64>,
 <tf.Variable 'forward/1A/layer0_normalization/beta:0' shape=(100,) dtype=float64>,
 <tf.Variable 'forward/1A/layer0_normalization/gamma:0' shape=(100,) dtype=float64>,
 <tf.Variable 'forward/1A/layer1/Matrix:0' shape=(100, 100) dtype=float64>,
 <tf.Variable 'forward/1A/layer1/normalization/beta:0' shape=(100,) dtype=float64>,
 <tf.Variable 'forward/1A/layer1/normalization/gamma:0' shape=(100,) dtype=float64>,
 <tf.Variable 'forward/1A/layer2/Matrix:0' shape=(100, 100) dtype=float64>,
 <tf.Variable 'forward/1A/layer2/normalization/beta:0' shape=(100,) dtype=float64>,
 <tf.Variable 'forward/1A/layer2/normalization/gamma:0' shape=(100,) dtype=float64>,
 <tf.Variable 'forward/1A/final/Matrix:0' shape=(100, 100) dtype=float64>,
 <tf.Variable 'forward/1A/final/normalization/beta:0' sh

In [32]:
name_list2 = []
for f in trainable_variables:
    name_list2.append(f.name)

In [33]:
len(name_list2)

422

In [34]:
name_list2

['Y0:0',
 'Z0:0',
 'Gamma0:0',
 'A0:0',
 'forward/1A/layer0_normalization/beta:0',
 'forward/1A/layer0_normalization/gamma:0',
 'forward/1A/layer1/Matrix:0',
 'forward/1A/layer1/normalization/beta:0',
 'forward/1A/layer1/normalization/gamma:0',
 'forward/1A/layer2/Matrix:0',
 'forward/1A/layer2/normalization/beta:0',
 'forward/1A/layer2/normalization/gamma:0',
 'forward/1A/final/Matrix:0',
 'forward/1A/final/normalization/beta:0',
 'forward/1A/final/normalization/gamma:0',
 'forward/1Gamma/layer0_normalization/beta:0',
 'forward/1Gamma/layer0_normalization/gamma:0',
 'forward/1Gamma/layer1/Matrix:0',
 'forward/1Gamma/layer1/normalization/beta:0',
 'forward/1Gamma/layer1/normalization/gamma:0',
 'forward/1Gamma/layer2/Matrix:0',
 'forward/1Gamma/layer2/normalization/beta:0',
 'forward/1Gamma/layer2/normalization/gamma:0',
 'forward/1Gamma/final/Matrix:0',
 'forward/1Gamma/final/normalization/beta:0',
 'forward/1Gamma/final/normalization/gamma:0',
 'forward/2A/layer0_normalization/beta:0

In [21]:
numpy.finfo(numpy.float32).max

3.4028235e+38

In [22]:
numpy.finfo(numpy.float32).min

-3.4028235e+38